# Notebook under development to import from excel and ReqIF to Capella model.

In [1]:
import capellambse
import populate_requirements_helper
import ipywidgets as widgets
from IPython.core.display import HTML
import pandas as pd

#String processing does like objects of type None. This is case when you pull content expecting to be a string and None is found.
def convert_nonetype (instr):
    if instr is None :
        instr = ""
        return instr
    else:
        return instr


resources = {
    "Trail_Power_for_Campgrounds": "Trail_Power_for_Campgrounds/Trail Power for Campgrounds",
}
path_to_model = "../Trail Power.aird"
model = capellambse.MelodyModel(path_to_model, resources=resources)
module_name = "Fred"
try :
    module = model.sa.requirement_modules.by_name(module_name)
except:
  print("Module :", modulename," not Found")



In [2]:
#String processing does like objects of type None. This is case when you pull content expecting to be a string and None is found.
def convert_nonetype (instr):
    if instr is None :
        instr = ""
        return instr
    else:
        #this fixes any encoding errors. 
        instr_bytes = instr.encode('utf-8')
        instr_str = instr_bytes.decode('utf-8')
        return instr_str

def sortnumber(outline_number):
    temp = outline_number.replace('-','.')
    ischapter = temp == outline_number 
    if ischapter :

        listofnumbers = temp.split('.')
        outlineforsort = ''
        for i in range(len(listofnumbers)) :
            listofnumbers[i] = listofnumbers[i].zfill(10)
        for temp in listofnumbers :
            outlineforsort = outlineforsort + temp + '.' 
        #pad for up to 10 header letters  .0000000000   
        #01.00.00.00.00.00
        #01.00.00.00.00.00
        #01.01.00.00.00.00
        # find out number of headers
        # add padding for up to 20 heading level
        levels = outlineforsort.count('.')
        padded_levels = 20 - levels
        for i in range(padded_levels):
            outlineforsort = outlineforsort + "0000000000."    
    else :
        #split into to parts x.x.x-x.x.x  first part is outline second is requirements place 
        outline_requirement = outline_number.split('-')
        print( outline_requirement[0])
        print( outline_requirement[1])
        listofnumbers = outline_requirement[0].split('.')
        outlineforsort = ''
        for i in range(len(listofnumbers)) :
            listofnumbers[i] = listofnumbers[i].zfill(10)
        for temp in listofnumbers :
            outlineforsort = outlineforsort + temp + '.'
        #01.00.00.00.00.00
        #01.00.00.R0.00.00   
        #01.00.00.R0.R1.00
        #01.00.00.R0.R2.00
        #01.00.00.R1.R0.00
        #01.01.00.00.00.00
        # find out number of headers
        # add padding for up to 10 heading levels for chapters
        levels = outlineforsort.count('.')
        padded_levels = 10 - levels
        for i in range(padded_levels):
            outlineforsort = outlineforsort + "0000000000."  
            
                
        listofnumbers = outline_requirement[1].split('.')
        req_outlineforsort = ''
        for i in range(len(listofnumbers)) :
            listofnumbers[i] = 'R' + listofnumbers[i].zfill(9)
        #concat the to parts
        for temp in listofnumbers :
            req_outlineforsort =  req_outlineforsort + temp + '.'
        #01.00.00.00.00.00
        #01.00.00.R0.00.00
        #01.00.00.R0.R1.00
        #01.00.00.R0.R2.00
        #01.00.00.R1.R0.00
        #01.01.00.00.00.00
        # find out number of headers for the requirement 
        # add padding for up to 10 heading level for requirement
        levels = req_outlineforsort.count('.')
        padded_levels = 10 - levels
        for i in range(padded_levels):
            req_outlineforsort = req_outlineforsort + "0000000000." 
        outlineforsort = outlineforsort +  req_outlineforsort
        
    return outlineforsort

def type_id(outline_number):
    temp = outline_number.replace('-','.')
    ischapter = temp == outline_number 
    if ischapter :
        this_type_id = "heading"    
    else:
        this_type_id = "systemrequirement"
    return this_type_id
    
def chapternumberplus1R(outline_number):
    temp = outline_number.replace('-','.')
    ischapter = temp == outline_number 
    if ischapter :
        listofnumbers = temp.split('.')
        outlineforsort = ''
        for i in range(len(listofnumbers)) :
            listofnumbers[i] = listofnumbers[i].zfill(10)
        for temp in listofnumbers :
            outlineforsort = outlineforsort + temp + '.'      
    else :
        #split into to parts x.x.x-x.x.x   first part is outline second is requirements 
        outline_requirement = outline_number.split('-')
        print( outline_requirement[0])
        print( outline_requirement[1])
        listofnumbers = outline_requirement[0].split('.')
        outlineforsort = ''
        for i in range(len(listofnumbers)) :
            listofnumbers[i] = listofnumbers[i].zfill(10)
        for temp in listofnumbers :
            outlineforsort = outlineforsort + temp + '.'
        listofnumbers = outline_requirement[1].split('.')
        req_outlineforsort = ''
        for i in range(len(listofnumbers)) :
            listofnumbers[i] = 'R' + listofnumbers[i].zfill(9)
            break;
        #concat the to parts
        for temp in listofnumbers :
            req_outlineforsort =  req_outlineforsort + temp + '.'
            break;
        outlineforsort = outlineforsort + req_outlineforsort
        print('level outline',outlineforsort)    
    return outlineforsort
  



def Populate_Parent_As_ID(moduleframe, row , level, parent) :
    #use a mix of while and do loop pattern, exiting on last row
    #while moduleframe.loc[row]['level'] >= level :
    while row < len(moduleframe.index)    and  moduleframe.loc[row]['level'] >= level :
        #print("Row Level", moduleframe.loc[row]['level'])
        if   moduleframe.loc[row]['level'] == level :
            print("row at same Level as previous")
            moduleframe.at[row ,"parent"]=parent
            #moduleframe.at[row ,"parent_row"]=parent_row
            #Print Line below may fail due to odd character in the fullname. 
            #print("Set row:",row,"Set Parent ID:",moduleframe.at[row ,"parent"],"Set id:",moduleframe.loc[row]['id'])  
        if  moduleframe.at[row,"level"] == level + 1 :
            #print("row at next level")
            new_parent = moduleframe.loc[row-1]['id'] 
            #new_parent_row = row-1
            new_level = moduleframe.loc[row]['level']
            row = Populate_Parent_As_ID(moduleframe, row , new_level , new_parent )# 
            #print("returned to previous level")
        #if  moduleframe.at[row,"level"] > level + 1 :
            #print("skipping row..",row)
            #print(moduleframe.loc[row]['Fullname'])
        row = row + 1
        if row == len(moduleframe.index) :
            break
    row = row - 1 
    return row    


def NumberinLevel(moduleframe, row , level, numberinlevel) :
    # sets the number with in a level 
    #use a mix of while and do loop pattern, exiting on last row
    #while moduleframe.loc[row]['level'] >= level :
    while row < len(moduleframe.index)    and  moduleframe.loc[row]['level'] >= level :
        #print("row", row,"id", moduleframe.loc[row]['id'], row ,"Row Level", moduleframe.loc[row]['level'])
        if   moduleframe.loc[row]['level'] == level :
            
            moduleframe.at[row ,"numberinlevel"] = numberinlevel
            print("row", row,"id", moduleframe.loc[row]['id'], row ,"Row Level", moduleframe.loc[row]['level'],"numberinlevel:",numberinlevel)
            numberinlevel = 1 + numberinlevel
            #moduleframe.at[row ,"parent_row"]=parent_row
            #Print Line below may fail due to odd character in the fullname. 
            #print("Set row:",row,"Set Parent ID:",moduleframe.at[row ,"parent"],"Set id:",moduleframe.loc[row]['id'])  
        if  moduleframe.at[row,"level"] == level + 1 :
             
            #print("row at next level")
            #new_parent_row = row-1
            new_level = moduleframe.loc[row]['level']
            print("Calling recursively setting new level",new_level)
            row = NumberinLevel(moduleframe, row , new_level , 0 )# 
            #print("returned to previous level")
        #if  moduleframe.at[row,"level"] > level + 1 :
            #print("skipping row..",row)
            #print(moduleframe.loc[row]['Fullname'])
        row = row + 1
        if row == len(moduleframe.index) :
            break
    row = row - 1 
    return row    

In [3]:
from pprint import pprint 
from lxml import etree
def Import_Requirements_from_MagicDraw_ReqIF(file_name):
    def pp(e):
        print (etree.tostring(e, pretty_print=True))
    
    
    
    # Load and parse the REQIF XML file and the schema file
    reqif_file_path = file_name
    
    
    
    tree = etree.parse(reqif_file_path)
    root = tree.getroot()
    
    # Print the root element
    print(f"Root element: {root.tag}")
    
    # Define the namespaces (update if needed based on your REQIF file)
    namespaces = {
        'default': 'http://www.omg.org/spec/ReqIF/20110401/reqif.xsd',
        'ns2': 'http://www.w3.org/1999/xhtml'
    }
    
    
    print("namespaces:",namespaces)
    
    # Extract CORE-CONTENT using namespaces if present
    core_content = root.find('.//default:CORE-CONTENT', namespaces)
    if core_content is not None:
    
        Spec_Type_df = pd.DataFrame(
             {
            'identifier': [],
            'lastchange': [], 
            'longname': [],
            }) 
        
    
        spec_types =  core_content.findall('.//default:SPECIFICATION-TYPE', namespaces)
        for spec_type in  spec_types:
            obj_id = spec_type.get('IDENTIFIER', 'N/A')
            obj_last_change = spec_type.get('LAST-CHANGE', 'N/A')
            obj_long_name= spec_type.get('LONG-NAME', 'N/A')
            new_row = {'identifier' : obj_id ,
                       'lastchange' : obj_last_change,
                       'longname' : obj_long_name } 
            #print(new_row)
            Spec_Type_df.loc[len(Spec_Type_df)] = new_row
            
        display(Spec_Type_df)
    
        Spec_Object_Type_df = pd.DataFrame(
            {
            'identifier': [],
            'lastchange': [], 
            'longname': [],
            }) 
        spec_object_types =  core_content.findall('.//default:SPEC-OBJECT-TYPE', namespaces)
        for spec_object_type in  spec_object_types:
            obj_id = spec_object_type.get('IDENTIFIER', 'N/A')
            obj_last_change = spec_object_type.get('LAST-CHANGE', 'N/A')
            obj_long_name= spec_object_type.get('LONG-NAME', 'N/A')
            new_row = {'identifier' : obj_id ,
                       'lastchange' : obj_last_change,
                       'longname' : obj_long_name} 
            #print(new_row)
            Spec_Object_Type_df.loc[len( Spec_Object_Type_df)] = new_row
            
        display(Spec_Object_Type_df)
    
    
        Req_Spec = pd.DataFrame(
             {
            'outlineNumber': [],
            'level': [], 
            'sortNumber': [],
            'last update': [], 
            'id': [], 
            'type_id': [],
            'Fullname': [],
            'description': [] ,
            'parent': [] 
            })    
        
        spec_objects = core_content.findall('.//default:SPEC-OBJECT', namespaces)
    
        print("Requirements:")
    
        for spec_object in spec_objects:
            #pprint(spec_object)
            obj_id = spec_object.get('IDENTIFIER', 'N/A')
            #long_name_elem = spec_object.find('.//default:LONG-NAME', namespaces)
            long_name_elem = spec_object.get('LONG-NAME', 'N/A')
            description_elem = spec_object.find('.//default:DESC', namespaces)
            last_change_elem = spec_object.get('LAST-CHANGE', 'N/A')
            #long_name = long_name_elem.text if long_name_elem is not None else 'N/A'
            description = description_elem.text if description_elem is not None else 'N/A'
    
            print(f"- ID: {obj_id}")
            print(f"  Long Name: {long_name_elem}")
            print(f"  Last Change: {last_change_elem}")
        
            values = spec_object.findall('.//default:VALUES', namespaces)
            for value in values:
                #pp(value)
                htmls = value.findall('.//default:ATTRIBUTE-VALUE-XHTML', namespaces)
                for html in htmls:
                    #pp(html)
                    html_values = value.findall('.//default:THE-VALUE', namespaces)
                    for html_value in html_values:
                        #pp(html_value)
                        desc_element = html_value.find('.//ns2:div', namespaces)
                        if desc_element is not None:
                            print(f"  HTML Text: {desc_element.text}")
                        else:
                            print("  HTML Text element not found.")
                            desc_element = ""
                    
                         
                section_numbers = value.findall('.//default:ATTRIBUTE-VALUE-STRING', namespaces)
                for section_number in section_numbers:  
                    section_number_element = section_number.get('THE-VALUE', 'N/A')
                print(f"  Section Number: {section_number_element}")   
                 
            #spec_type_identifiers = spec_object.findall('.//default:TYPE', namespaces)
            #for spec_type_identifier in spec_type_identifiers:  
            # print("spec type identifier:",spec_type_identifier.find('SPEC-OBJECT-TYPE-REF'))
            
            
            ##print the etree
            #pp(spec_object)
            #spec_object_type_ref = spec_object.find('.//SPEC-OBJECT/SPEC-OBJECT-TYPE-REF')
            spec_object_type_ref = spec_object.find('.//default:SPEC-OBJECT-TYPE-REF', namespaces)
            #pp(spec_object_type_ref)
            
            # Check if the element is found and extract the text
            if spec_object_type_ref is not None:
                mdreqif_value = spec_object_type_ref.text
                print(f"  Extracted value: {mdreqif_value}")
            else:
                print("  SPEC-OBJECT-TYPE-REF tag not found.")
            print()     
                
            new_row = {'outlineNumber' : section_number_element ,
                       'sortNumber' : sortnumber("0" ) ,
                       'Fullname' : long_name_elem,
                       'type_id': Spec_Object_Type_df.loc[Spec_Object_Type_df['identifier'] ==  mdreqif_value, 'longname'].values[0],
                       'description': desc_element.text,
                       'last update': last_change_elem ,
                       'id':  obj_id,
                       'level': chapternumberplus1R(convert_nonetype(section_number_element)).count('.') ,
                       'parent': "TBD" } 
            #print(new_row)
            Req_Spec.loc[len(Req_Spec)] = new_row      
    
    else:
        print("No CORE-CONTENT found in the REQIF file.")
    return Req_Spec
              


In [4]:
from openpyxl import *   
def Import_Requirements_from_Excel(file_name,sheet_name):

    exceldf = pd.read_excel(file_name,sheet_name)

    
    #print(exceldf.to_string(index=True))
          
    Req_Spec = pd.DataFrame(
         {
        'outlineNumber': [],
        'level': [], 
        'sortNumber': [],
        'last update': [], 
        'id': [], 
        'type_id': [],
        'Fullname': [],
        'description': [],
        'parent' : []
        })   
    
    for index, row in exceldf.iterrows():
        #print("Excel ID" , exceldf.loc[index]['Requirement ID'])
        #looks for id in SMW content
        full_name = '[' + exceldf.loc[index]['Requirement ID']+ ']' + ' '  + exceldf.loc[index]['Name'] 
        new_row = {'outlineNumber' : exceldf.loc[index]['Outline Number'] ,
                   'sortNumber' : sortnumber(exceldf.loc[index]['Outline Number'] ) ,
                   'Fullname' : full_name  , 'title' : exceldf.loc[index]['Name'],
                   'type_id': type_id(exceldf.loc[index]["Type"]),
                   'description': exceldf.loc[index]['Description'],
                   'last update': exceldf.loc[index]["Last Update"],
                   'id': exceldf.loc[index]['Requirement ID'],
                   'level': chapternumberplus1R(convert_nonetype(exceldf.loc[index]['Outline Number'])).count('.'),
                   'parent': "TBD" }  
        #print(new_row)
        Req_Spec.loc[len(Req_Spec)] = new_row
              
    
    #Pol_Req_Spec = pd.DataFrame(list(map(extract_attrs_of_interest, work_items)))
    return Req_Spec

In [5]:

#Req_Spec = Import_Requirements_from_ReqIF('TestTemplate_2.reqif')
Req_Spec = Import_Requirements_from_MagicDraw_ReqIF('braking_system.reqif')
display(Req_Spec)

#Req_Spec = Import_Requirements_from_Excel("Requirements.xlsx","Sheet1")
#display(Req_Spec)

Root element: {http://www.omg.org/spec/ReqIF/20110401/reqif.xsd}REQ-IF
namespaces: {'default': 'http://www.omg.org/spec/ReqIF/20110401/reqif.xsd', 'ns2': 'http://www.w3.org/1999/xhtml'}


,identifier,lastchange,longname
0,polarion_live_document,2013-01-01T00:00:00Z,Polarion Live Document


,identifier,lastchange,longname
0,_4ab9533c-20b2-48d8-9ba6-2abf2c83a062,2024-09-09T12:06:04.513-07:00,Heading
1,_53a5dd58-c6ec-44ce-9a1a-b4e0e4c80f8b,2024-09-09T12:06:04.516-07:00,System Requirement


Requirements:
- ID: rmf-f63d5ba9-a23a-46f0-b0b3-28a3ce93aecb
  Long Name: N/A
  Last Change: 2024-09-09T12:06:07.090-07:00
  Section Number: Introduction
  Extracted value: _4ab9533c-20b2-48d8-9ba6-2abf2c83a062



UnboundLocalError: cannot access local variable 'desc_element' where it is not associated with a value

In [ ]:
Req_Spec.to_excel ( 'export_Req_Specspec1.xlsx', index = False, header=True)  
Req_Spec = Req_Spec.sort_values(by='sortNumber')
Req_Spec = Req_Spec.reset_index(drop=True)  # make sure indexes pair with number of rows
Req_Spec = Req_Spec.sort_values(by='sortNumber')
Req_Spec = Req_Spec.reset_index(drop=True)  # make sure indexes pair with number of row
row = 0
level = 1
xlsx_file_name = 'export_Req_Specspec2.xlsx'
print("Excel for Req_Spec Spec post parent processing",xlsx_file_name)
Req_Spec.to_excel ( xlsx_file_name, index = False, header=True)  

row = Populate_Parent_As_ID(Req_Spec, row , level, "Spec")

#check the dataframe before processing

xlsx_file_name = 'export_Req_Specspec3.xlsx'
print("Excel for Req_Spec Spec post parent processing",xlsx_file_name)
Req_Spec.to_excel ( xlsx_file_name, index = False, header=True)  





def create_module_item(Req_Spec_DocItem, parent_module):
    #this code is dependent on the python library capellambse

    try:
        #create requitement sibling
        #print("Creating module item Req_Spec Type:",Req_Spec_DocItem['type_id'])
        if Req_Spec_DocItem['type_id'] == "heading" :

            temp_string_1 = '<p>'+convert_nonetype(Req_Spec_DocItem['id']) + ": " + convert_nonetype(Req_Spec_DocItem['Fullname'] )+'</p>'
            temp_string_2 = '<p>Updated: '+ convert_nonetype(Req_Spec_DocItem['last update']) + '</p>'
            temp_string_3 = '<p>'+ convert_nonetype(Req_Spec_DocItem['description']) + '</p>'
            temp_string = temp_string_1  + temp_string_2 + temp_string_3
            moduleObject = parent_module.folders.create(name=  convert_nonetype(Req_Spec_DocItem['Fullname']),
                prefix = Req_Spec_DocItem['id'] ,
                long_name = convert_nonetype(Req_Spec_DocItem['outlineNumber']) + " " + convert_nonetype(Req_Spec_DocItem['id']) + " " + convert_nonetype(Req_Spec_DocItem['Fullname']),
                description = temp_string
            )
            
            
        else:

            temp_string_1 = '<p>'+convert_nonetype(Req_Spec_DocItem['id']) + ": " + convert_nonetype(Req_Spec_DocItem['Fullname'] )+'</p>'
            temp_string_2 = '<p>Updated: '+ convert_nonetype(Req_Spec_DocItem['last update']) + '</p>'
            temp_string_3 = '<p>'+ convert_nonetype(Req_Spec_DocItem['description']) + '</p>'
            temp_string = temp_string_1  + temp_string_2 + temp_string_3
            moduleObject = parent_module.requirements.create(name = convert_nonetype(Req_Spec_DocItem['Fullname']),
                prefix = Req_Spec_DocItem['id'] ,
                long_name = convert_nonetype(Req_Spec_DocItem['outlineNumber']) + " " + convert_nonetype(Req_Spec_DocItem['id']) + " " + convert_nonetype(Req_Spec_DocItem['Fullname']),
                description = temp_string 
            )
           
                
    except:
        # if something went wrong we rollback the transaction
        raise
    else:
        # if everything is ok we commit the transaction
        model.save()

    
    # save the Capella model
    
    return moduleObject

#print(module) 
#help(module)

In [ ]:
if module.requirements == [] :
    
    #start creation of content in the model
    
    
    try:
        #change name of Module to name of Req_Spec Document
        module.long_name = Req_Spec.loc[0]['Fullname']
       
                
    except:
        # if something went wrong we rollback the transaction
        raise
    else:
        # if everything is ok we commit the transaction
         model.save()
        #print("Model Committed")
    
    # save the Capella model
    #print("Module Name;",module.get_name())
    #delete the row with document information        

    
    #Funtion to create a structure in Capella/SMW when nothing exits previously 
    # it returns a row in table to support recursion
    def create_requirement_module_object(Source_Req_DF, moduleparent , row, current_level):
        """
        Populates a SMW module with a requirement from a row in the dataframe 
        
        Args:
                Source_Req_Dc : Panda data frame with rows to be populated.  
                moduleparent : The is the parent that requirement will be added as a child.
                row : this is the row in  Req_Spec being processed.
                current_level : This is the level in structure that requirement are being created.
                
        #this is a recursive algorithm designed to produce content in document level
        #the procedure invoke itself when it detects indention change in document
        #it will loop at a level while the document ends and greater than equal to when called.
        #use a modified do loop pattern
        """
        new_module_parent = moduleparent
        while   row < len( Source_Req_DF.index)    and  Source_Req_DF.loc[row]['level'] >= current_level :
            Source_Req_DF_Row = Source_Req_DF.loc[row]
            #print("Row:",row)
            print('Source Item ID', Source_Req_DF_Row['id'])
            print('Source Item Outline',Source_Req_DF_Row['outlineNumber'])
            Source_Req_DF_Row_level = Source_Req_DF_Row['level']
            print('Source Item Level', Source_Req_DF_Row_level) 
            print('current_level', current_level) 
            if Source_Req_DF_Row_level == current_level: 
                print("creating new module object")
                new_module_parent = create_module_item(Source_Req_DF_Row, moduleparent )
                
            if Source_Req_DF_Row_level == current_level + 1: 
                # since spec is indenting a one level  
                print("switched to new parent module",new_module_parent)
                print("new parent  name",new_module_parent.name)
                row =  create_requirement_module_object(Source_Req_DF,  new_module_parent  , row, Source_Req_DF_Row_level)
            row = row + 1
            if row == len(Source_Req_DF.index) :
                break
            if Source_Req_DF_Row_level > current_level + 1:
                #this code may not be needed. It would require a Req_Spec document to skip a level of indention to encoutner it. It could be removed if deemed impossible.
                #print("Skipping row:", row ,"Title:" ,Req_Spec_DocItem['title'], "due to unexpected change to a indention of to",Req_Spec_DocItem_level)
                #it will be reflected as a singel level
                row =  create_requirement_module_object(Source_Req_DF,  new_module_parent  , row, Source_Req_DF_Row_level)
                row = row + 1;
        # go back a row since it was incremented and return 
        row = row - 1       
        return row



    
    #Processing inports... 
    starting_level = 0
    starting_row = 0
    #print("Number of Rows", len(Req_Spec.index))
    row = create_requirement_module_object(Req_Spec, module , starting_row, starting_level)
    #print("Last row",row)
    
    ##



                
                
                
             
             
         
            
                               


    

   
        
    
    